# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006, and
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro)
1. [Step 1](#rderiv): ${\rm d}r / {\rm d}t$
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant.

We start with the following parameters:
1. a Cartesian tortoise separation vector $\left( x, y, z \right)$ (in units of total mass),
1. a Cartesian momentum vector ${\bf p^{*}}$ (in units of reduced mass),
1. a spin vector ${\bf S}_{1}$ (in units of total mass squared),
1. a spin vector ${\bf S}_{2}$ (in units of total mass squared),
1. orbital phase (in radians), and
1. $\alpha \cdot \cos\left( \iota \right)$ where $\iota$ is the inclination of the system.

We include below the physical parameters necessary to compute the initial conditions.

In [1]:
import sympy as sp
m1,m2,x,y,z,S1x,S1y,S1z,S2x,S2y,S2z = sp.symbols("m1 m2 x y z S1x S1y S1z S2x S2y S2z",real=True)

gamma = 0.577215664901532860606512090082402431

<a id='rderiv'></a>

# Step ?: ${\rm d}r / {\rm d}t$ \[Back to [top](#toc)\]
$$\label{rderiv}$$

<font color='red'>We begin our initial condition journey at LALSimIMRSpinPrecEOB.c Line 1328, where the orbital phase and $\alpha \cdot \cos \left( \iota \right)$ are set to zero.  The previous code in "Step 1" of this file is for the "SpinsAlmostAligned" case.  After setting these values, we move to LALSpinPrecHcapRvecDerivative_v3opt.c where we compute the derivatives of the separation vector.</font>

<a id='mass_radius_terms'></a>

# ${\rm d}r / {\rm d}t$ \[Back to [top](#toc)\]
$$\label{mass_radius_terms}$$

<font color='red'>LALSpinPrecHcapRvecDerivative_v3opt.c Line 15 claims that we're first going to compute [BB2010](https://arxiv.org/abs/0912.3517) Equation (A4), but I don't believe that's the case.</font>

We need the following quantities:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \mu &= \frac{ m_{1} m_{2} }{ M } \\
    \eta &= \frac{ \mu }{ M } \\
    r^{2} &= x^{2} + y^{2} + z^{2} \\
    r &= \sqrt{r^{2}} \\
    u &= \frac{ 1 }{ r }
\end{align*}

In [2]:
M = m1 + m2
mu = m1*m2/M
eta = mu/M
xi = 1
rsq = x*x + y*y + z*z
r = sp.sqrt(rsq)
u = 1/r

<a id='skerr'></a>

# ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }
\end{equation*}

so that

\begin{equation*}
    a = \sqrt{ a^{2} }
\end{equation*}

In [3]:
SKerr1 = S1x + S2x
SKerr2 = S1y + S2y
SKerr3 = S1z + S2z
asq = (SKerr1*SKerr1 + SKerr2*SKerr2 + SKerr3*SKerr3)/(M*M)
a = sp.sqrt(asq)

<a id='w'></a>

# $\varpi$ \[Back to [top](#toc)\]
$$\label{w}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.7)

\begin{equation*}
    \varpi^{2} = r^{2} + a^{2}.
\end{equation*}

In [4]:
w2 = rsq + asq

<a id='Dinv'></a>

# $D^{-1}$ \[Back to [top](#toc)\]
$$\label{Dinv}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.82)

\begin{equation*}
    D^{-1} = 1 + \log \left[ 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3} \right].
\end{equation*}

In [5]:
Dinv = 1 + sp.log(1 + 6*eta*u*u + 2*(26 - 3*eta)*eta*u*u*u)

<a id='k'></a>

# $K$ \[Back to [top](#toc)\]
$$\label{k}$$

<font color='red'>We see $K$ used ("KK") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 65, before it's calculuated on Line 86.</font>

Following the discussion in the two paragrahs after [BB2010](https://arxiv.org/abs/0912.3517) Equation (6.11), we have

\begin{equation*}
    K = 1.4467 \left( 1 - 4 \eta \right)^{2} + 4 \left( 1 - 2 \eta \right) \eta.
\end{equation*}

In [6]:
K = 1.4467*(1 - 4*eta)*(1 - 4*eta) + 4*(1 - 2*eta)*eta

<a id='deltaubar'></a>

# $\bar{\Delta}_{u}$ \[Back to [top](#toc)\]
$$\label{deltaubar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.75)

\begin{equation*}
    \bar{\Delta}_{u} = \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K -1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }.
\end{equation*}

In [7]:
Deltaubar = asq*u*u/(M*M) + 2*u/(eta*K - 1) + 1/((eta*K - 1)*(eta*K - 1))

<a id='calibcoeffs'></a>

# Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ \[Back to [top](#toc)\]
$$\label{calibcoeffs}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.77)--(5.81), we have

\begin{align*}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{0}\\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} \right.\\
        &\left.\ \ \ \ \ + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align*}

<font color='red'>There are two more coefficients in the code that are not listed in [BB2010](https://arxiv.org/abs/0912.3517), and whose origin I have not been able to trace.</font>

\begin{align*}
    \Delta_{5} &= \\
    \Delta_{5l} &= 
\end{align*}

In [8]:
Delta0 = K*(eta*K - 2)
Delta1 = -2*(eta*K - 1)*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a*a*(eta*K - 1)*(eta*K - 1)*Delta0/(M*M)
Delta3 = sp.Rational(1,3)*(-Delta1**3 + 3*(eta*K - 1)*Delta1**2 + 3*Delta2*Delta1 - 6*(eta*K - 1)*(-eta*K + Delta2 + 1)
                           - 3*a*a*(eta*K - 1)**2*Delta1/(M*M))
Delta4 = sp.Rational(1,12)*(6*a*a*(Delta1**2 - 2*Delta2)*(eta*K - 1)**2/(M*M) + 3*Delta1**4 - 8*(eta*K - 1)*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*(eta*K - 1)*Delta2 + Delta3)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*sp.pi**2)*(eta*K - 1)**2
                           + 6*(Delta2**2 - 4*Delta3*(eta*K - 1)))
Delta5 = (eta*K - 1)**2*(-sp.Rational(4237,60) + sp.Rational(128,5)*gamma
                          + 2275*sp.pi*sp.pi*sp.Rational(1,512)
                          - sp.Rational(1,3)*a*a*(Delta1**3 - 3*Delta1*Delta2 + 3*Delta3)
                          - (Delta1**5 - 5*Delta1**3*Delta2 + 5*Delta1*Delta2**2 + 5*Delta1**2*Delta3
                             - 5*Delta2*Delta3 - 5*Delta1*Delta4)*sp.Rational(1,5)/((eta*K - 1)**2)
                          + (Delta1**4 - 4*Delta1**2*Delta2 + 2*Delta2**2 + 4*Delta1*Delta3 - 4*Delta4)
                          *sp.Rational(1,2)/(eta*K - 1) + sp.Rational(256,5)*sp.log(2))
Delta5l = sp.Rational(64,5)*(eta*K - 1)**2

<a id='deltau'></a>

# $\Delta_{u}$ \[Back to [top](#toc)\]
$$\label{deltau}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.73)

\begin{equation*}
    \Delta_{u} = \bar{\Delta}_{u} \left[ 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \right].
\end{equation*}

<font color='red'>Note that in LALSpinPrechcapRvecDerivative_v3opt.c line 77 we take an absolute value.</font>

In [9]:
Deltau = Deltaubar*(1 + eta*Delta0 + eta*sp.log(1 + Delta1*u + Delta2*u*u + Delta3*u*u*u + Delta4*u*u*u*u))

<a id='deltat'></a>

# $\Delta_{t}$ \[Back to [top](#toc)\]
$$\label{deltat}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.71)

\begin{equation*}
    \Delta_{t} = r^{2} \Delta_{u}.
\end{equation*}

In [10]:
Deltat = r*r*Deltau

<a id='deltar'></a>

# $\Delta_{r}$ \[Back to [top](#toc)\]
$$\label{deltar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.38)

\begin{equation*}
    \Delta_{r} = \Delta_{t} D^{-1}.
\end{equation*}

In [11]:
Deltar = Deltat*Dinv

<a id='xi'></a>

# $\xi$ \[Back to [top](#toc)\]
$$\label{xi}$$

Inverting [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (28),

\begin{equation*}
    \xi = \frac{ \sqrt{ \Delta_{t} \Delta_{r} } }{ \varpi^{2} }.
\end{equation*}

In [12]:
xi = sp.sqrt(Deltat*Deltar)/w2

<a id='constants'></a>

# Constants \[Back to [top](#toc)\]
$$\label{constants}$$

<font color='red'>We now move to LALSimIMRCalculateSpinPrecEOBHCoeffs.c to recompute the coefficients.  In Lines 77--84, the following are listed as "constants are fits taken from PRD 86, 024011 (2012) E. 37".  That [reference](https://arxiv.org/abs/1202.0790) does not, however, contain either that many constants or that many significant digits...  On Lines 98 and 99, we see coefficients b3 and bb3 set to zero.  There is a note to check whether b3 and bb3 can ever be nonzero; if that question was answered, there is no record of it.</font>

Below, we will denote c0 by $c_{0}$, and similarly for the other constants.

In [13]:
c0  = 1.4467
c1  = -1.7152360250654402
c2  = -3.246255899738242
c20  = 1.712
c21  = -1.803949138004582
c22  = -39.77229225266885
c23  = 103.16588921239249

b3 = 0.
bb3 = 0.0

<a id='k2'></a>

# $K$ (redo) \[Back to [top](#toc)\]
$$\label{k2}$$

<font color='red'>We see $K$ used ("KK") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 65, before it's calculuated on Line 86.  It is NOT clear that the computations coincid; the question of WHY this recomputation takes place is open.</font>

We now re-compute $K$ (which makes no sense, because it's a constant dependent only on $\eta$).  Since v3 always takes SpinAlignedEOBversion = 2, on LALSpinPrecHcapRvecDerivative_v3opt.c Line 102 we have

\begin{equation*}
    K = c_{20} + c_{21} \eta + c_{22} \eta^{2} + c_{23} \eta^{3}.
\end{equation*}

In [14]:
K = c20 + c21*eta + c22*eta*eta + c23*eta*eta*eta

<a id='calibcoeffs2'></a>

# Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ (redo) \[Back to [top](#toc)\]
$$\label{calibcoeffs2}$$

<font color='red'>We see these coefficients used ("ki") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 71, before they're calculuated on Line 86.  It IS clear that these computations agree with the previous computations; the question of WHY this recomputation takes place is open.</font>

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.77)--(5.81), we have

\begin{align*}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{0}\\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} \right.\\
        &\left.\ \ \ \ \ + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align*}

<font color='red'>There are two more coefficients in the code that are not listed in [BB2010](https://arxiv.org/abs/0912.3517), and whose origin I have not been able to trace.</font>

\begin{align*}
    \Delta_{5} &= \\
    \Delta_{5l} &= 
\end{align*}

In [15]:
Delta0 = K*(eta*K - 2)
Delta1 = -2*(eta*K - 1)*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a*a*(eta*K - 1)*(eta*K - 1)*Delta0/(M*M)
Delta3 = sp.Rational(1,3)*(-Delta1**3 + 3*(eta*K - 1)*Delta1**2 + 3*Delta2*Delta1 - 6*(eta*K - 1)*(-eta*K + Delta2 + 1)
                           - 3*a*a*(eta*K - 1)**2*Delta1/(M*M))
Delta4 = sp.Rational(1,12)*(6*a*a*(Delta1**2 - 2*Delta2)*(eta*K - 1)**2/(M*M) + 3*Delta1**4 - 8*(eta*K - 1)*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*(eta*K - 1)*Delta2 + Delta3)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*sp.pi**2)*(eta*K - 1)**2
                           + 6*(Delta2**2 - 4*Delta3*(eta*K - 1)))
Delta5 = (eta*K - 1)**2*(-sp.Rational(4237,60) + sp.Rational(128,5)*gamma
                          + 2275*sp.pi*sp.pi*sp.Rational(1,512)
                          - sp.Rational(1,3)*a*a*(Delta1**3 - 3*Delta1*Delta2 + 3*Delta3)
                          - (Delta1**5 - 5*Delta1**3*Delta2 + 5*Delta1*Delta2**2 + 5*Delta1**2*Delta3
                             - 5*Delta2*Delta3 - 5*Delta1*Delta4)*sp.Rational(1,5)/((eta*K - 1)**2)
                          + (Delta1**4 - 4*Delta1**2*Delta2 + 2*Delta2**2 + 4*Delta1*Delta3 - 4*Delta4)
                          *sp.Rational(1,2)/(eta*K - 1) + sp.Rational(256,5)*sp.log(2))
Delta5l = sp.Rational(64,5)*(eta*K - 1)**2

<a id='calibconsts'></a>

# Calibration Constants \[Back to [top](#toc)\]
$$\label{calibconst}$$

<font color='red'>From LALSimIMRCalculateSpinPrecEOBHCoeffs.c Lines 127--130 we have</font>

\begin{align*}
    d_{1} &= -69.5 \\
    d_{1v2} &= -74.71 - 156 \eta + 627.5 \eta^{2} \\
    d_{H_{\rm eff}\ \rm SS} &= 2.75 \\
    d_{H_{\rm eff}\ \rm SSv2} &= 8.127 - 154.2 \eta + 830.8 \eta^{2}.
\end{align*}

<font color='red'>This marks the end of the CalculateSpinPrecEOB routine.</font>

In [16]:
d1 = -69.5
d1v2 = -sp.Rational(7471,100) - 156*eta + sp.Rational(1255,2)*eta*eta
dheffSS = sp.Rational(11,4)
dheffSSv2 = sp.Rational(8127,1000) - sp.Rational(771,5)*eta + sp.Rational(4154,5)*eta*eta

<a id='tortoise_matrix'></a>

# Tortoise Conversion Matrix \[Back to [top](#toc)\]
$$\label{tortoise_matrix}$$

<font color='red'>We're now back to LALSpinPrecHcapRvecDerivative_v3opt.c, Lines 92--96.</font>

From [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (A3) the matrix for the coordinate conversion to tortoise coordinates is

\begin{align*}
    \begin{pmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{pmatrix}
\end{align*}

In [17]:
ximinus1 = xi - 1
toTort = sp.Array([[1 + x*x*ximinus1/(r*r), x*y*ximinus1/(r*r), x*z*ximinus1/(r*r)],
                   [x*y*ximinus1/(r*r), 1 + y*y*ximinus1/(r*r), y*z*ximinus1/(r*r)],
                   [x*z*ximinus1/(r*r), y*z*ximinus1/(r*r), 1 + z*z*ximinus1/(r*r)]])

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [18]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Initial_Conditions.ipynb to latex
[NbConvertApp] Writing 61076 bytes to Tutorial-SEOBNR_Initial_Conditions.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
